# Домашнее задание к лекции "Основы веб-скрапинга и работы с API"

## Задание 1. 

### Обязательная часть

Будем парсить страницу со свежеми новостям на [habr.com/ru/all/](https://habr.com/ru/all/).

Вам необходимо собирать только те статьи, в которых встречается хотя бы одно требуемое ключевое слово. Эти слова определяем в начале кода в переменной, например:

`KEYWORDS = ['python', 'парсинг']`

 Поиск вести по всей доступной preview-информации (это информация, доступная непосредственно с текущей страницы). 
 
В итоге должен формироваться датафрейм вида: `<дата> - <заголовок> - <ссылка>`

### Дополнительная часть (необязательная)

Улучшить скрипт так, чтобы он анализировал не только preview-информацию статьи, но и весь текст статьи целиком.

Для этого потребуется получать страницы статей и искать по тексту внутри этой страницы.  

Итоговый датафрейм формировать со столбцами: `<дата> - <заголовок> - <ссылка> - <текст_статьи>`


In [850]:
import requests
import pandas as pd
import time
from bs4 import BeautifulSoup

In [710]:
KEYWORDS = ['python', 'парсинг']

URL = 'https://habr.com/ru/all/'

In [711]:
req = requests.get(URL)
soup = BeautifulSoup(req.text, 'html.parser')

In [712]:
#  Находим количество новостных страниц

num_pages = soup.find('div',class_="tm-pagination__pages").text
pages = [int(item) for item in num_pages.replace('...', '').split()]
last_page = pages[-1]
last_page

50

In [713]:
#  Список ссылок на новостные страницы

list_urls = [str(URL + 'page'+ str(i)) for i in range(1, last_page+1)]
# list_urls

In [732]:
#  TEST: Тестовый список ссылок на новостные страницы
list_urls = [str(URL + 'page'+ str(i)) for i in range(1, 17)]
list_urls

['https://habr.com/ru/all/page1',
 'https://habr.com/ru/all/page2',
 'https://habr.com/ru/all/page3',
 'https://habr.com/ru/all/page4',
 'https://habr.com/ru/all/page5',
 'https://habr.com/ru/all/page6',
 'https://habr.com/ru/all/page7',
 'https://habr.com/ru/all/page8',
 'https://habr.com/ru/all/page9',
 'https://habr.com/ru/all/page10',
 'https://habr.com/ru/all/page11',
 'https://habr.com/ru/all/page12',
 'https://habr.com/ru/all/page13',
 'https://habr.com/ru/all/page14',
 'https://habr.com/ru/all/page15',
 'https://habr.com/ru/all/page16']

##### Вариант 1

In [734]:
def get_all_links(urls):
    """
    Функция получения ссылок на полные статьи по всем страницам новостей
    page number = 50
    """
    all_refs = []
    for url in urls: 
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')

        time.sleep(0.3)

        posts = soup.find_all('article', class_='tm-articles-list__item')
        post_h2 = set()
        post_h3 = set()


        if soup.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2'):
            post_h2 = set(map(lambda x: x.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2'), posts))
            if None in post_h2:
                post_h2.remove(None)

        if soup.find('div', class_='tm-megapost-snippet__tint'):
            post_h3 = set(map(lambda x: x.find('div', class_='tm-megapost-snippet'), posts))
            if None in post_h3:
                post_h3.remove(None)

        posts_list = list(post_h2) + list(post_h3)
        post_link = list(map(lambda x: x.find('a').get('href'), posts_list))
        link_list = list(map(lambda x: 'https://habr.com' + x, post_link))
        all_refs +=link_list

    return all_refs

In [735]:
def get_news_table(links):
     """
    Поиск новостей, в которых встречается хотя бы одно требуемое ключевое слово из KEYWORDS,
    по всей доступной preview-информации
    """
    news_ = pd.DataFrame()
    for link in links:
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        time.sleep(0.3)
        
        text = soup.find('div', class_='article-formatted-body').text.lower()

        for word in KEYWORDS:
            if word in text:
                date = pd.to_datetime(soup.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime'), dayfirst=True)
                title = soup.find('h1', class_='tm-article-snippet__title').text

                row = {'date': date, 'title': title, 'link': link, 'text': text}
                news_ = pd.concat([news_, pd.DataFrame([row])])
                # Прекращаем парсить статью, если обнаружено ключевое слово. Предотвращение создания дубликатов
                break

    news_table = news_.reset_index()
    news_table.drop('index', axis=1, inplace=True)
    return news_table

In [736]:
get_news_table(get_all_links(list_urls))

,date,title,link,text
0,2021-08-05 12:12:11+00:00,Интеграция поисковой машины Яндекса с сайтом. ...,https://habr.com/ru/post/571428/,за 3 месяца сразу 2 клиента интерволги (интерн...
1,2021-08-05 10:54:22+00:00,Трек DeepPavlov в онлайн-школе Community of Op...,https://habr.com/ru/company/samsung/blog/571404/,"лето подходит к концу, а возможность поработат..."
2,2021-08-05 10:12:10+00:00,Где работать в ИТ в 2021: GigAnt,https://habr.com/ru/company/habr_career/blog/5...,сегодня в рубрике «где работать в ит» расскаже...
3,2021-08-05 07:40:02+00:00,Приглашаем компании к спонсорcтву PyCon Russia...,https://habr.com/ru/company/it_people/blog/571...,если спросить опытного питониста в каких компа...
4,2021-08-05 05:11:33+00:00,История одного фееричного провала тестового за...,https://habr.com/ru/post/571342/,"просидев на одном предприятии несколько лет, я..."
5,2021-08-05 05:18:28+00:00,Спектральный анализ временных рядов с помощью ...,https://habr.com/ru/post/571344/,с развитием информационных технологий професси...
6,2021-08-05 08:05:38+00:00,Ускоряем код на Питоне с помощью расширений на Cи,https://habr.com/ru/company/alconost/blog/571366/,производительность си — в программах на питоне...
7,2021-08-04 16:40:44+00:00,Разбираемся с ChainMap из коллекций Python,https://habr.com/ru/company/skillfactory/blog/...,chainmap — инструмент управления поиском в сло...
8,2021-08-04 15:14:11+00:00,Формируем реляционную модель из schemaless баз...,https://habr.com/ru/company/otus/blog/571286/,mongodb – одна из самых популярных документ-ор...
9,2021-08-04 15:36:23+00:00,Ансамблевые методы машинного обучения,https://habr.com/ru/post/571296/,ансамблевые методы - это мощный инструмент для...


In [737]:
d = get_news_table(get_all_links(list_urls))
d[['text']]

,text
0,за 3 месяца сразу 2 клиента интерволги (интерн...
1,"лето подходит к концу, а возможность поработат..."
2,сегодня в рубрике «где работать в ит» расскаже...
3,если спросить опытного питониста в каких компа...
4,"просидев на одном предприятии несколько лет, я..."
5,с развитием информационных технологий професси...
6,производительность си — в программах на питоне...
7,chainmap — инструмент управления поиском в сло...
8,mongodb – одна из самых популярных документ-ор...
9,ансамблевые методы - это мощный инструмент для...


## Вариант 2: 
Тестирование ошибок:
- AttributeError: 'NoneType' object has no attribute 'text' - 20 страница последний пост https://habr.com/ru/company/kingston_technology/blog/569952/


In [633]:
list_urls = [str(URL + 'page'+ str(i)) for i in range(1, last_page+1)]

# list_urls

In [797]:
list_urls = [str(URL + 'page'+ str(i)) for i in range(20, 21)]

# (list_urls)

In [800]:
def get_all_links(urls):
    """
    Функция получения ссылок на полные статьи на 
    """ 
    all_refs = []
    
    for url in urls: 
        res = requests.get(url)
        soup = BeautifulSoup(res.text, 'html.parser')

        time.sleep(0.3)

        posts = soup.find_all('article', class_='tm-articles-list__item')
        post_h2 = set()
        post_h3 = set()


        if soup.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2'):
            post_h2 = set(map(lambda x: x.find('h2', class_='tm-article-snippet__title tm-article-snippet__title_h2'), posts))
            if None in post_h2:
                post_h2.remove(None)

        if soup.find('div', class_='tm-megapost-snippet__tint'):
            post_h3 = set(map(lambda x: x.find('div', class_='tm-megapost-snippet'), posts))
            if None in post_h3:
                post_h3.remove(None)

        posts_list = list(post_h2) + list(post_h3)
        post_link = list(map(lambda x: x.find('a').get('href'), posts_list))
        link_list = list(map(lambda x: 'https://habr.com' + x, post_link))
        all_refs +=link_list
        
    return all_refs

In [803]:
def get_news_table(links):
    news_ = pd.DataFrame()
    
    for link in links:
        soup = BeautifulSoup(requests.get(link).text, 'html.parser')
        time.sleep(0.3)
        
        text = soup.find(class_='article-formatted-body').text.lower()

        for word in KEYWORDS:
            if word in text:
                date = pd.to_datetime(soup.find('span', class_='tm-article-snippet__datetime-published').find('time').get('datetime'), dayfirst=True)
                title = soup.find('h1', class_='tm-article-snippet__title').text

                row = {'date': date, 'title': title, 'link': link, 'text': text}
                news_ = pd.concat([news_, pd.DataFrame([row])])
                # Прекращаем парсить статью, если обнаружено ключевое слово. Предотвращение создания дубликатов
                break

    news_table = news_.reset_index()
    news_table.drop('index', axis=1, inplace=True)
    return news_table

In [804]:
# Pages 20
get_news_table(get_all_links(list_urls))

AttributeError: 'NoneType' object has no attribute 'text'

In [770]:
# Pages 1-10
get_news_table(get_all_links(list_urls))

,date,title,link,text
0,2021-08-05 12:12:11+00:00,Интеграция поисковой машины Яндекса с сайтом. ...,https://habr.com/ru/post/571428/,за 3 месяца сразу 2 клиента интерволги (интерн...
1,2021-08-05 10:54:22+00:00,Трек DeepPavlov в онлайн-школе Community of Op...,https://habr.com/ru/company/samsung/blog/571404/,"лето подходит к концу, а возможность поработат..."
2,2021-08-05 10:12:10+00:00,Где работать в ИТ в 2021: GigAnt,https://habr.com/ru/company/habr_career/blog/5...,сегодня в рубрике «где работать в ит» расскаже...
3,2021-08-05 07:40:02+00:00,Приглашаем компании к спонсорcтву PyCon Russia...,https://habr.com/ru/company/it_people/blog/571...,если спросить опытного питониста в каких компа...
4,2021-08-05 05:11:33+00:00,История одного фееричного провала тестового за...,https://habr.com/ru/post/571342/,"просидев на одном предприятии несколько лет, я..."
5,2021-08-05 05:18:28+00:00,Спектральный анализ временных рядов с помощью ...,https://habr.com/ru/post/571344/,с развитием информационных технологий професси...
6,2021-08-05 08:05:38+00:00,Ускоряем код на Питоне с помощью расширений на Cи,https://habr.com/ru/company/alconost/blog/571366/,производительность си — в программах на питоне...
7,2021-08-04 16:40:44+00:00,Разбираемся с ChainMap из коллекций Python,https://habr.com/ru/company/skillfactory/blog/...,chainmap — инструмент управления поиском в сло...
8,2021-08-04 15:14:11+00:00,Формируем реляционную модель из schemaless баз...,https://habr.com/ru/company/otus/blog/571286/,mongodb – одна из самых популярных документ-ор...
9,2021-08-04 15:36:23+00:00,Ансамблевые методы машинного обучения,https://habr.com/ru/post/571296/,ансамблевые методы - это мощный инструмент для...


In [790]:
# Pages 11-19
get_news_table(get_all_links(list_urls))

,date,title,link,text
0,2021-08-02 10:14:52+00:00,Кратко: запросы к API и разбор XML-ответов. Py...,https://habr.com/ru/post/570852/,этот пост предназначен в первую очередь для но...
1,2021-08-02 12:03:46+00:00,Распознавание блюд в кафетерии банка,https://habr.com/ru/company/alfa/blog/569208/,в кафетерии альфа-банка в обеденный час-пик об...
2,2021-08-02 11:22:06+00:00,"Метеостанция на Banana Pi M64 (Linux, C#, Dock...",https://habr.com/ru/company/timeweb/blog/569748/,\n\r\nстатей о создании метеостанции на базе a...
3,2021-08-02 09:13:04+00:00,Немного про современные технологии Greybox-фаз...,https://habr.com/ru/company/bizone/blog/570312/,\nавтор: иннокентий сенновский\nкак найти баги...
4,2021-08-02 06:58:01+00:00,10 приниципов разработки на Java,https://habr.com/ru/post/570654/,"сколько раз вы были в ситуации, когда вы хотел..."
5,2021-08-02 07:00:02+00:00,"Еще одна стажировка, или сказ про реверс IP-ка...",https://habr.com/ru/company/ntc-vulkan/blog/56...,"вступлениеитак, меня взяли на стажировку. стаж..."
6,2021-08-01 16:24:05+00:00,Установка JUNIPER VRR через CLI на гипервизоре...,https://habr.com/ru/post/570752/,в определенный момент домашняя лаба (бесшумная...
7,2021-08-01 17:48:35+00:00,FOSS News №81 – дайджест материалов о свободно...,https://habr.com/ru/post/570764/,\n\nвсем привет!\nпродолжаем дайджесты новосте...
8,2021-08-01 11:12:39+00:00,Когда нужна телеметрия: интегрируем в проект б...,https://habr.com/ru/post/570720/,"два сигнала телеметрии, формируемые в микрокон..."
9,2021-08-01 17:34:10+00:00,Чем машинное обучение отличается от статистики...,https://habr.com/ru/company/skillfactory/blog/...,к старту курса о машинном и глубоком обучении ...


In [750]:
# Test for one link


## Задание 2.

### Обязательная часть

Написать скрипт, который будет проверять список e-mail адресов на утечку при помощи сервиса [Avast Hack Ckeck](https://www.avast.com/hackcheck/).
Список email-ов задаем переменной в начале кода:  
`EMAIL = [xxx@x.ru, yyy@y.com]`

В итоге должен формироваться датафрейм со столбцами: `<дата утечки> - <источник утечки> - <описание утечки>`  

**Подсказка**: сервис работает при помощи "скрытого" API. Внимательно изучите post-запросы.

In [867]:
import json

In [868]:
EMAIL = ['xxx@x.ru', 'yyy@y.com']
emails = {
    'emailAddresses' : EMAIL
}

URL = 'https://identityprotection.avast.com/v1/web/query/site-breaches/unauthorized-data'

headers = {
    'Vaar-Header-App-Build-Version': '1.0.0', 
    'Vaar-Header-App-Product': 'hackcheck-web-avast',
    'Vaar-Version': '0',
    'Vaar-Header-App-Product-Name': 'hackcheck-web-avast'
}

In [881]:
request = requests.post(URL, json=emails, headers=headers)
request.json()

{'breaches': {'2': {'breachId': 2,
   'site': 'linkedin.com',
   'recordsCount': 158591429,
   'description': "In 2012, online professional networking platform LinkedIn fell victim to a breach of its members' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \n\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.",
   'publishDate': '2016-10-21T00:00:00Z',
   'statistics': {'usernames': 0,
    'passwords': 111975337,
    'emails': 158591429}},
  '3': {'breachId': 3,
   'site': 'adobe.com',
   'recordsCount': 152046506,
   'description': "In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software produc

In [884]:
# Не хватает 2х утечек?! утечки по обоим мейлам на сайтах - adobe.com, imesh.com

breaches = pd.DataFrame(request.json()['breaches']).T.reset_index()

In [883]:
breaches[['publishDate', 'site', 'description']]

,publishDate,site,description
0,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
1,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
2,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
3,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
4,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...
5,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
6,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
7,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
8,2017-03-15T00:00:00Z,globalreach.eu,"In 2016, Global Reach Technology's database wa..."
9,2017-03-01T00:00:00Z,rayli.com.cn,"On an unconfirmed date, Chinese gossip site Ra..."


In [886]:
# В breaches хранится сайт на кот. произошла утечка, время утечки и описание

breaches = request.json()['breaches']
breaches

{'2': {'breachId': 2,
  'site': 'linkedin.com',
  'recordsCount': 158591429,
  'description': "In 2012, online professional networking platform LinkedIn fell victim to a breach of its members' passwords. Four years later, 117 million email and password combinations from that breach appeared for sale on a dark web marketplace. \n\nThe leaked passwords had only been protected by unsalted SHA-1 cryptographic hashing, which prompted LinkedIn to enforce salted hashing after the breach. Russian national Yevgeniy Nikulin was accused of the breach and was extradited from the Czech Republic to the United States as of March 2018.",
  'publishDate': '2016-10-21T00:00:00Z',
  'statistics': {'usernames': 0, 'passwords': 111975337, 'emails': 158591429}},
 '3': {'breachId': 3,
  'site': 'adobe.com',
  'recordsCount': 152046506,
  'description': "In October of 2013, criminals penetrated Adobe's corporate network and the stole source code for several of its software products. The affected products incl

In [888]:
# нет полезной информации

data = request.json()['data']

In [876]:
# почта по которой произошла утечка

summary = request.json()['summary']
summary

{'xxx@x.ru': {'breaches': [3, 12, 15, 2961, 3164, 3176]},
 'yyy@y.com': {'breaches': [2,
   3,
   15,
   41,
   3520,
   3587,
   3669,
   13094,
   13662,
   16768,
   17009,
   17110,
   17670,
   37177]}}

In [887]:
leak_df = pd.DataFrame()

for mail, breaches in request.json()['summary'].items():

    for index in breaches['breaches']:
#         print(type(index))
        index = str(index)
        site = request.json()['breaches'][index]['site']
        description = request.json()['breaches'][index]['description']
        date = request.json()['breaches'][index]['publishDate']
        rows = {'mail' : mail, 'date' : date, 'site' : site, 'description' :  description}
        leak_df = pd.concat([leak_df, pd.DataFrame([rows])])
        
leak_df.reset_index(drop=True)

,mail,date,site,description
0,xxx@x.ru,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
1,xxx@x.ru,2016-10-29T00:00:00Z,vk.com,Popular Russian social networking platform VKo...
2,xxx@x.ru,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
3,xxx@x.ru,2017-01-31T00:00:00Z,cdprojektred.com,"In March 2016, CDProjektRed.com.com's forum da..."
4,xxx@x.ru,2017-02-14T00:00:00Z,cfire.mail.ru,"In July and August of 2016, two criminals carr..."
5,xxx@x.ru,2017-02-14T00:00:00Z,parapa.mail.ru,"In July and August 2016, two criminals execute..."
6,yyy@y.com,2016-10-21T00:00:00Z,linkedin.com,"In 2012, online professional networking platfo..."
7,yyy@y.com,2016-10-21T00:00:00Z,adobe.com,"In October of 2013, criminals penetrated Adobe..."
8,yyy@y.com,2016-10-23T00:00:00Z,imesh.com,"In June 2016, a cache of over 51 million user ..."
9,yyy@y.com,2016-10-24T00:00:00Z,dropbox.com,Cloud storage company Dropbox suffered a major...


### Дополнительная часть (необязательная)

Написать скрипт, который будет получать 50 последних постов указанной группы во Вконтакте.  
Документация к API VK: https://vk.com/dev/methods
, вам поможет метод [wall.get](https://vk.com/dev/wall.get)  
```
GROUP = 'netology'  
TOKEN = УДАЛЯЙТЕ В ВЕРСИИ ДЛЯ ПРОВЕРКИ, НА GITHUB НЕ ВЫКЛАДЫВАТЬ  
```

В итоге должен формироваться датафрейм со столбцами: `<дата поста> - <текст поста>`

#### ПРИМЕЧАНИЕ
Домашнее задание сдается ссылкой на репозиторий [GitHub](https://github.com/).
Не сможем проверить или помочь, если вы пришлете:
- файлы;
- архивы;
- скриншоты кода.

Все обсуждения и консультации по выполнению домашнего задания ведутся только на соответствующем канале в slack.

##### Как правильно задавать вопросы аспирантам, преподавателям и коллегам?
Прежде чем задать вопрос необходимо попробовать найти ответ самому в интернете. Навык самостоятельного поиска информации – один из важнейших, и каждый практикующий специалист любого уровня это делает каждый день.

Любой вопрос должен быть сформулирован по алгоритму:  
1) Что я делаю?  
2) Какого результата я ожидаю?  
3) Как фактический результат отличается от ожидаемого?  
4) Что я уже попробовал сделать, чтобы исправить проблему?  

По возможности, прикрепляйте к вопросу скриншоты, либо ссылки на код. Оставляйте только проблемный и воспроизводимый участок кода, все решение выкладывать не допускается.
